In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains

import time
from datetime import datetime

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [ ]:
driver = Chrome(executable_path='/Users/chrisjohnson/Downloads/chromedriver')
url = "https://www.radissonhotels.com/en-us/destination"
driver.get(url)
time.sleep(10)
# viewhotels_button = driver.find_elements_by_xpath('/html/body/section/pacsys-animation[3]/pacsys-directory/div[3]/button')[0]
# viewhotels_button.click()
# northamhotels_button = driver.find_elements_by_xpath('/html/body/section/pacsys-animation[3]/pacsys-directory/div[2]/pacsys-directory-listings/div/div[5]/a')[0]
# northamhotels_button.click()

In [ ]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

In [ ]:
stuff = soup.find('div', {'id' : 'index-5'})

In [ ]:
cities = stuff.find_all('li', {'class' : 'mb-2'})
citieslist = []
for x in cities:
    city = {}
    city['name'] = x.find('a').text
    city['url'] = x.find('a').attrs['href']
    citieslist.append(city)
    print(city)
    print('='*50)
cities_df = pd.DataFrame(citieslist)
cities_df

In [ ]:
cities_df.to_csv('radisson_cities.csv')

In [ ]:
cities_df = pd.read_csv('radisson_cities.csv')

In [ ]:
driver = Chrome(executable_path='/Users/chrisjohnson/Downloads/chromedriver')
hotellist = []
for index, row in cities_df.iterrows():
    cityurl = 'https://www.radissonhotels.com/' + row['url']
    try:
        driver.get(cityurl)
        city_source = driver.page_source
        citysoup = BeautifulSoup(city_source, 'lxml')
        stuff = citysoup.find_all('div', {'class' : 'container'})
        hotels = stuff[2].find_all('a')
        for x in hotels:
            hotel = {}
            hotel['name'] = x.find('h3').text
            hotel['url'] = x.attrs['href']
            hotellist.append(hotel)
            print(hotel)
            print('='*50)
    except:
        pass
hotels_df = pd.DataFrame(hotellist)
hotels_df

In [3]:
hotels_df = pd.read_csv('radisson_urls.csv')
hotels_df

,Unnamed: 0,name,url
0,0,"Country Inn & Suites by Radisson, Dothan, AL",/en-us/hotels/country-inn-dothan-al
1,1,"Country Inn & Suites by Radisson, Homewood, AL",/en-us/hotels/country-inn-homewood-al
2,2,"Country Inn & Suites by Radisson, Madison, AL",/en-us/hotels/country-inn-madison-al
3,3,"Country Inn & Suites by Radisson, Monroeville, AL",/en-us/hotels/country-inn-monroeville-al
4,4,"Country Inn & Suites by Radisson, Montgomery E...",/en-us/hotels/country-inn-montgomery-al
5,5,"Country Inn & Suites by Radisson, Saraland, AL",/en-us/hotels/country-inn-saraland-al
6,6,"Country Inn & Suites by Radisson, Tuscaloosa, AL",/en-us/hotels/country-inn-tuscaloosa-al
7,7,"Country Inn & Suites by Radisson, Chandler, AZ",/en-us/hotels/country-inn-chandler-az
8,8,"Country Inn & Suites by Radisson, Flagstaff Do...",/en-us/hotels/country-inn-flagstaff-downtown-az
9,9,"Country Inn & Suites by Radisson, Flagstaff, AZ",/en-us/hotels/country-inn-flagstaff-az


In [59]:
def get_stuff(dataframe):
    driver = Chrome(executable_path='/Users/chrisjohnson/Downloads/chromedriver')
    dataframe = dataframe
    counter = 0
    hoteladdresses = []
    allhoteldescriptions = []
    hotelratings = []
    for index, row in dataframe.iterrows():
        
        name = row['name']  
        hotelurl = 'https://www.radissonhotels.com' + row['url']
        
        try:
            driver.get(hotelurl)
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'lxml')
#             print(soup)
            
            try:
                addresssoup = soup.find_all('a', {'class', 't-full-address'})
                for x in addresssoup:
                    address = x.text.replace('\n','').replace('\xa0','').strip()
                    address = address.split(',')
                    hoteladdress = {}
                    hoteladdress['name'] = name
                    hoteladdress['url'] = hotelurl
                    hoteladdress['street'] = address[0]
                    hoteladdress['locality'] = address[1]
                    hoteladdress['state'] = address[2][:-5]
                    hoteladdress['postal'] = address[2][-5:]
                    hoteladdresses.append(hoteladdress)
                print(hoteladdress)
                print()
                print(datetime.now())
                print('='*80)
                print('='*80)
                print()
            except:
                hoteladdress = {}
                hoteladdress['name'] = name
                hoteladdress['url'] = hotelurl
                hoteladdress['street'] = 'N/A'
                hoteladdress['city'] = 'N/A'
                hoteladdress['state'] = 'N/A'
                hoteladdress['postal'] = 'N/A'
                hoteladdresses.append(hoteladdress)
                print(hoteladdress)
                print()
                print(datetime.now())
                print('='*80)
                print('='*80)
                print()
            
            try:
                descriptions = soup.find_all('div', {'class', 'content-overflowed'})
                hoteldescription = {}
                hoteldesc = ''
                for x in descriptions:
                    x = ' ' + x.text.replace('\n','')
                    hoteldesc += x
                hoteldescription['name'] = name
                hoteldescription['description'] = hoteldesc
                allhoteldescriptions.append(hoteldescription)
                print(hoteldescription)
                print()
                print(datetime.now())
                print('='*80)
                print('='*80)
                print()
            except:
                hoteldescription = {}
                hoteldescription['name'] = name
                hoteldescription['description'] = 'N/A'
                allhoteldescriptions.append(hoteldescription)
                print(hoteldescription)
                print()
                print(datetime.now())
                print('='*80)
                print('='*80)
                print()
            try:
                hotelrating = {}
                hotelrating['name'] = name
                hotelrating['rating'] = soup.find('img', {'class' : 'tripadvisor-rating'}).attrs['src']
                hotelratings.append(hotelrating)
                print(hotelrating)
                print()
                print(datetime.now())
                print('='*80)
                print('='*80)
                print()
            except:
                hotelrating = {}
                hotelrating['name'] = name
                hotelrating['rating'] = 'N/A'
                hotelratings.append(hotelrating)
                print(hotelrating)
                print()
                print(datetime.now())
                print('='*80)
                print('='*80)
                print()
        except:
            hoteladdress = {}
            hoteladdress['name'] = name
            hoteladdress['url'] = hotelurl
            hoteladdresses.append(hoteladdress)
        counter += 1
        print(f'Attempted {counter} hotels.')
    hotel_df = pd.DataFrame(hoteladdresses)
    desc_df = pd.DataFrame(allhoteldescriptions)
    rating_df = pd.DataFrame(hotelratings)
    merged = pd.merge(hotel_df, desc_df)
    merged = pd.merge(merged, rating_df)
    return  merged

In [60]:
radisson = get_stuff(hotels_df)

{'name': 'Country Inn & Suites by Radisson, Dothan, AL', 'url': 'https://www.radissonhotels.com/en-us/hotels/country-inn-dothan-al', 'street': '3465 Ross Clark Circle', 'locality': '                                        Dothan', 'state': 'Alabama', 'postal': '36303'}

2020-09-25 10:50:46.470046

{'name': 'Country Inn & Suites by Radisson, Dothan, AL', 'description': ' Conveniently located on travel routes, the Country Inn & Suites by Radisson, Dothan, AL provides an easy rest stop with a variety of comfortable rooms and suites. Looking for great golf? We’re just 15 minutes from the Robert Trent Jones Golf Trail championship courses at Highland Oaks. Families who stay at our hotel often visit Water World Water Park, historic Landmark Park, and The Carver Museum among many nearby attractions.When you arrive, treat yourself to complimentary cookies and a cup of coffee before you settle into your room and log into free Wi-Fi. Our business center lets you take care of office tasks 24 hour

{'name': 'Country Inn & Suites by Radisson, Monroeville, AL', 'url': 'https://www.radissonhotels.com/en-us/hotels/country-inn-monroeville-al', 'street': '120 Highway 21 S', 'locality': '                                        Monroeville', 'state': 'Alabama', 'postal': '36460'}

2020-09-25 10:50:57.923703

{'name': 'Country Inn & Suites by Radisson, Monroeville, AL', 'description': " Located near the Literary Capital of Alabama, the Country Inn & Suites by Radisson, Monroeville, AL welcomes guests with Southern hospitality and boasts an excellent location for exploring local sites. Discover attractions that inspired Harper Lee's writings, like the Old Courthouse Museum and the Faulk Property, or spend the afternoon on the water at Claiborne Lake. Corporate visitors appreciate easy access from our hotel to nearby businesses like Georgia-Pacific and Harrigan Lumber Company.Back at the hotel, unwind by our outdoor pool and stay in touch with the office using free Wi-Fi. In the mornings, a

{'name': 'Country Inn & Suites by Radisson, Tuscaloosa, AL', 'url': 'https://www.radissonhotels.com/en-us/hotels/country-inn-tuscaloosa-al', 'street': '4801 McFarland Boulevard E', 'locality': '                                        Tuscaloosa', 'state': 'Alabama', 'postal': '35405'}

2020-09-25 10:51:06.953455

{'name': 'Country Inn & Suites by Radisson, Tuscaloosa, AL', 'description': ' Cheer on the Crimson Tide at the University of Alabama, see a concert at Tuscaloosa Amphitheater, or enjoy a riverboat cruise on the Black Warrior River, all within minutes of the Country Inn & Suites by Radisson, Tuscaloosa, AL. Our hotel’s location off I-20/59 and I-359 offers easy access to UA for prospective students taking a campus tour and families visiting current students. Business travelers appreciate that the hotel is also close to Michelin, Nucor Steel, and Mercedes-Benz.After a day of adventure, grab a book and head outdoors to relax by the pool. Need to reenergize after a long drive? Enj

{'name': 'Country Inn & Suites by Radisson, Flagstaff, AZ', 'url': 'https://www.radissonhotels.com/en-us/hotels/country-inn-flagstaff-az', 'street': '3501 E Lockett Road', 'locality': '                                        Flagstaff', 'state': 'Arizona', 'postal': '86004'}

2020-09-25 10:51:15.311792

{'name': 'Country Inn & Suites by Radisson, Flagstaff, AZ', 'description': " Pull off Historic Route 66 and settle into your spacious room or suite at the Country Inn & Suites by Radisson, Flagstaff, AZ. Our hotel is ideally located for day trips to the Grand Canyon and is just minutes from the beautiful sights of the Walnut Canyon National Monument. Visit your favorite student at Northern Arizona University, just seven miles away, or treat yourself to a day of shopping at Flagstaff Mall or in Historic Downtown Flagstaff, both less than four miles away. Science buffs can plan a trip to Lowell Observatory, and outdoor-lovers enjoy the scenic views at Coconino National Forest.\xa0After ch

{'name': 'Country Inn & Suites by Radisson, Page, AZ', 'rating': 'https://www.tripadvisor.com/img/cdsi/img2/ratings/traveler/4.5-MCID-5.svg'}

2020-09-25 10:51:24.467094

Attempted 12 hotels.
{'name': 'Country Inn & Suites by Radisson, Phoenix Airport, AZ', 'url': 'https://www.radissonhotels.com/en-us/hotels/country-inn-phoenix-airport-az', 'street': '4702 E University Drive', 'locality': '                                        Phoenix', 'state': 'Arizona', 'postal': '85034'}

2020-09-25 10:51:28.153196

{'name': 'Country Inn & Suites by Radisson, Phoenix Airport, AZ', 'description': " Fly into Phoenix Sky Harbor International Airport (PHX) and take the complimentary shuttle to the Country Inn & Suites by Radisson, Phoenix Airport, AZ. Located between downtown Scottsdale and downtown Phoenix, our hotel is also minutes away from Arizona State University in Tempe. You can see an Arizona Diamondbacks game at Chase Field, attend a concert at Talking Stick Resort Arena, and visit monkeys a

...

{'name': 'Country Inn & Suites by Radisson, Appleton North, WI', 'url': 'https://www.radissonhotels.com/en-us/hotels/country-inn-little-chute-wi', 'street': '130 Patriot Drive', 'locality': '                                        Little Chute', 'state': 'Wisconsin', 'postal': '54140'}

2020-09-25 11:21:27.279639

{'name': 'Country Inn & Suites by Radisson, Appleton North, WI', 'description': " If you're visiting the Appleton area, reserve a room at the Country Inn & Suites by Radisson, Appleton North, WI to stay just off I-41 and only five miles from Lawrence University. Our hotel in Little Chute is conveniently located five miles from downtown Appleton, where you can find attractions like The Trout Museum of Art and The Building for Kids children's museum. If you're in the mood for shopping, stop by the popular Fox River Mall, home to more than 180 stores.After a busy day of exploring Appleton, return to the hotel to unwind at our indoor water park. Kids can play on the waterslide an

{'name': 'Radisson Hotel Madison', 'url': 'https://www.radissonhotels.com/en-us/hotels/radisson-madison', 'street': '517 Grand Canyon Drive', 'locality': '                                        Madison', 'state': 'Wisconsin', 'postal': '53719'}

2020-09-25 11:21:40.112391

{'name': 'Radisson Hotel Madison', 'description': ' Located off Highway 12/14, Radisson Hotel Madison offers easy commutes to the University of Wisconsin and nearby corporations like Epic, CUNA, and University Research Park. If you’re in town for a family vacation, you’ll appreciate our proximity to the Henry Vilas Zoo and tours at Wisconsin’s State Capitol. Less than a mile away, West Towne Mall features more than 80 popular stores for all the shopping addicts, while Camp Randall Stadium, a 12-minute drive from our hotel, keeps Badgers football fans happy.When you’re not meeting clients or touring local attractions, recharge with on-site amenities like an indoor pool, a hot tub, and a fitness center, open 24 hours.

{'name': 'Country Inn & Suites by Radisson, Milwaukee Airport, WI', 'url': 'https://www.radissonhotels.com/en-us/hotels/country-inn-milwaukee-airport-wi', 'street': '6200 S 13th Street', 'locality': '                                        Milwaukee', 'state': 'Wisconsin', 'postal': '53221'}

2020-09-25 11:21:51.642191

{'name': 'Country Inn & Suites by Radisson, Milwaukee Airport, WI', 'description': ' Conveniently located off I-94, the Country Inn & Suites by Radisson, Milwaukee Airport, WI is just two miles south of General Mitchell International Airport (MKE). Cheer on the Brewers at Miller Park, 14 minutes from the hotel, or enjoy a day of family fun at the Milwaukee County Zoo, a short drive away. In town for a conference or trade show? The Wisconsin Center in the heart of downtown is flanked by some of the city’s finest restaurants and cultural spots like the Milwaukee Public Museum.Take advantage of our hotel’s complimentary shuttle for convenient service to and from the MKE ai

{'name': 'Country Inn & Suites by Radisson, Prairie du Chien, WI', 'url': 'https://www.radissonhotels.com/en-us/hotels/country-inn-prairie-du-chien-wi', 'street': "1801 Cabela's Lane", 'locality': '                                        Prairie du Chien', 'state': 'Wisconsin', 'postal': '53821'}

2020-09-25 11:22:06.682206

{'name': 'Country Inn & Suites by Radisson, Prairie du Chien, WI', 'description': " The Country Inn & Suites by Radisson, Prairie du Chien, WI is surrounded by the natural beauty of the upper Mississippi River. Guests appreciate our location for its easy access to Iowa, Wyalusing State Park, and Pikes Peak State Park. If you forgot to pack something, head next door to Cabela’s, which has everything you need to enjoy outdoor activities. Families with children love our indoor water park, Bear Foot Bay, which offers an activity pool with water basketball, a 25-foot waterslide, water geysers, and a waterfall spa. You can start your day with a complimentary, hot breakfa

{'name': 'Country Inn & Suites by Radisson, Mt. Pleasant-Racine West, WI', 'url': 'https://www.radissonhotels.com/en-us/hotels/country-inn-sturtevant-mt-pleasant-racine-wi', 'street': '13339 Hospitality Court', 'locality': '                                        Sturtevant', 'state': 'Wisconsin', 'postal': '53177'}

2020-09-25 11:22:18.404065

{'name': 'Country Inn & Suites by Radisson, Mt. Pleasant-Racine West, WI', 'description': " The Country Inn & Suites by Radisson, Mt. Pleasant-Racine West, Wisconsin is conveniently located off I-94/I-41 in Sturtevant less than 10 minutes from downtown Racine and approximately 20 minutes from General Mitchell International Airport (MKE). For business travelers, enjoy a quick commute from the hotel to business meetings at CNH Industrial and Bombardier Recreational Products. Vacationing with the family? Make memories at the Racine Zoo, Six Flags Great America, and Miller Park, home of the Milwaukee Brewers. You can also plan a day trip to downtown

In [61]:
radisson

,name,url,street,locality,state,postal,description,rating
0,"Country Inn & Suites by Radisson, Dothan, AL",https://www.radissonhotels.com/en-us/hotels/co...,3465 Ross Clark Circle,Dothan,Alabama,36303,"Conveniently located on travel routes, the Co...",https://www.tripadvisor.com/img/cdsi/img2/rati...
1,"Country Inn & Suites by Radisson, Homewood, AL",https://www.radissonhotels.com/en-us/hotels/co...,485 Wildwood North Circle,Homewood,Alabama,35209,"Set in a quiet neighborhood off I-65, the Cou...",https://www.tripadvisor.com/img/cdsi/img2/rati...
2,"Country Inn & Suites by Radisson, Madison, AL",https://www.radissonhotels.com/en-us/hotels/co...,101 Westchester Drive,Madison,Alabama,35758,"Located right off I-565, the Country Inn & Su...",https://www.tripadvisor.com/img/cdsi/img2/rati...
3,"Country Inn & Suites by Radisson, Monroeville, AL",https://www.radissonhotels.com/en-us/hotels/co...,120 Highway 21 S,Monroe...,Alabama,36460,"Located near the Literary Capital of Alabama,...",https://www.tripadvisor.com/img/cdsi/img2/rati...
4,"Country Inn & Suites by Radisson, Montgomery E...",https://www.radissonhotels.com/en-us/hotels/co...,5155 Carmichael Road,Montgo...,Alabama,36106,Situated just minutes from downtown Montgomer...,https://www.tripadvisor.com/img/cdsi/img2/rati...
5,"Country Inn & Suites by Radisson, Saraland, AL",https://www.radissonhotels.com/en-us/hotels/co...,1130 Joaneen Drive,Saraland,Alabama,36571,"Mobile, known as the Azalea City, is only a 1...",https://www.tripadvisor.com/img/cdsi/img2/rati...
6,"Country Inn & Suites by Radisson, Tuscaloosa, AL",https://www.radissonhotels.com/en-us/hotels/co...,4801 McFarland Boulevard E,Tuscal...,Alabama,35405,Cheer on the Crimson Tide at the University o...,https://www.tripadvisor.com/img/cdsi/img2/rati...
7,"Country Inn & Suites by Radisson, Chandler, AZ",https://www.radissonhotels.com/en-us/hotels/co...,7425 W Chandler Boulevard,Chandler,Arizona,85226,"Located off I-10, the Country Inn & Suites by...",https://www.tripadvisor.com/img/cdsi/img2/rati...
8,"Country Inn & Suites by Radisson, Flagstaff Do...",https://www.radissonhotels.com/en-us/hotels/co...,224 S Mikes Pike Street,Flagstaff,Arizona,86001,Whether you plan to hike Coconino National Fo...,https://www.tripadvisor.com/img/cdsi/img2/rati...
9,"Country Inn & Suites by Radisson, Flagstaff, AZ",https://www.radissonhotels.com/en-us/hotels/co...,3501 E Lockett Road,Flagstaff,Arizona,86004,Pull off Historic Route 66 and settle into yo...,https://www.tripadvisor.com/img/cdsi/img2/rati...


In [62]:
radisson.to_csv('radisson_descriptions.csv')